In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,LabelEncoder,RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
from sklearn.linear_model import LogisticRegression

In [4]:
df=pd.read_csv(r'C:\Users\pavan\Desktop\data science\decision tree\loan_data.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [6]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [7]:
x=df.drop('loan_status',axis = 1)
y = df.loan_status


In [8]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,random_state=42,train_size=0.8)


In [9]:
num_col = xtrain.select_dtypes(include='number').columns
cat_col = xtrain.select_dtypes(include ='object').columns

In [10]:
def cat_col_unique(cols):
    for col in cols:
        print(f'{col}-{df[col].nunique()}')

In [11]:
cat_col_unique(cat_col)

person_gender-2
person_education-5
person_home_ownership-4
loan_intent-6
previous_loan_defaults_on_file-2


In [12]:
preprocessing = ColumnTransformer(
    transformers = [
        ('scaler',RobustScaler(),num_col),
        ('onehotencoder',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),cat_col.drop('person_education')),
        ('Ordinal_encoder',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['person_education']),
    ]
)#preprocessing.fit_transform()

In [13]:
pipeline = Pipeline(
    steps =[
        ('preprocessing',preprocessing),
        ('model',DecisionTreeClassifier())
    ]
)

In [14]:
pipeline.fit(xtrain,ytrain)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scaler', RobustScaler(),
                                                  Index(['person_age', 'person_income', 'person_emp_exp', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'credit_score'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['person_gender', 'person_home_ownership', 'loan_intent',
       'previous_loan_defaults_on_file'],
      dtype='object')),
                                                 ('Ordinal_encoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['person_education'])])),
                ('model', DecisionTreeClassifier())])

In [15]:
print('train score: ',pipeline.score(xtrain,ytrain))
print('test score : ',pipeline.score(xtest,ytest))

train score:  1.0
test score :  0.9044444444444445


In [16]:
log_pipeline = Pipeline(
    steps =[
        ('preprocessing',preprocessing),
        ('model',LogisticRegression())
    ]
)
log_pipeline.fit(xtrain,ytrain)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scaler', RobustScaler(),
                                                  Index(['person_age', 'person_income', 'person_emp_exp', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'credit_score'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['person_gender', 'person_home_ownership', 'loan_intent',
       'previous_loan_defaults_on_file'],
      dtype='object')),
                                                 ('Ordinal_encoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['person_education'])])),
                ('model', LogisticRegression())])

In [17]:
t=log_pipeline.score(xtrain,ytrain)
t1=log_pipeline.score(xtest,ytest)
print('train score : ',t,'\ntest score : ',t1)

train score :  0.8973333333333333 
test score :  0.8937777777777778


In [18]:
#max_depth ---level of depth of leaf node, max_depth =-1 is default (reaches till last leaf node)
# splitter --- used to find the best split
# sub nodes are called as decision node
# min_Sample_leaf ---checks left n right
#min _sample_split =  
#max features== by default it none...to select the root node it will check for all features....randomly it select 1 root node


In [19]:
grid_search =GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid = {'criterion': ['gini', 'entropy'],
                'max_depth': [3, 5, 10, None],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 5],
                'class_weight': ['balanced', None]

    },
    cv = 5,
    n_jobs=-1,
    verbose = 3,
    scoring='f1_macro'
)

In [20]:
grid_search.fit(xtrain,ytrain)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


ValueError: 
All the 720 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\pavan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pavan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\pavan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\tree\_classes.py", line 1024, in fit
    super()._fit(
    ~~~~~~~~~~~~^
        X,
        ^^
    ...<2 lines>...
        check_input=

In [ ]:
grid_search.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
grid_search.score(xtrain,ytrain)

In [ ]:
model = grid_search.best_estimator_

In [ ]:
model.fit(xtrain,ytrain) 